In [1]:
import numpy as np

import astropy.units as u
from astropy.time import Time
from astropy.coordinates import solar_system_ephemeris

from poliastro.bodies import Sun,Venus, Earth, Jupiter, Saturn
from poliastro.twobody import Orbit
from poliastro.maneuver import Maneuver
from poliastro.iod import izzo
from poliastro.plotting import OrbitPlotter2D
from poliastro.util import norm

from scipy import optimize as opt
 
solar_system_ephemeris.set("jpl")

<ScienceState solar_system_ephemeris: 'jpl'>

In [13]:
cassini_venus2 = Time("1999-06-24")
cassini_earth2 = Time("1999-08-18")
cassini_jupiter = Time("2000-12-30")
cassini_saturn = Time("2004-07-01")

#cassini transit
cassini_js_time = cassini_saturn-cassini_jupiter
cassini_ej_time = cassini_jupiter-cassini_earth2
cassini_ve_time = cassini_earth2-cassini_venus2

In [32]:
tmod=[0,0,0,0]
plot_on=0;

def trajectory_calculator(t=[0,0,0,0],plot_on=0,disp_on=0):
    #start with cassini style assumptions
    cassini_venus2 = Time("1999-06-24")
    cassini_earth2 = Time("1999-08-18")
    cassini_jupiter = Time("2000-12-30")
    cassini_saturn = Time("2004-07-01")

    #cassini transit
    cassini_js_time = cassini_saturn-cassini_jupiter
    cassini_ej_time = cassini_jupiter-cassini_earth2
    cassini_ve_time = cassini_earth2-cassini_venus2


    tmod_s  = t[0] #arrival time in saturn relative to 2020
    tmod_js = t[1] #transit times
    tmod_ej = t[2] 
    tmod_ve = t[3]


    time_js = tmod_js*u.year
    time_ej = tmod_ej*u.year
    time_ve = tmod_ve*u.year

    date_arrive_saturn = Time("2020-01-01", scale="utc")+tmod_s*u.year;


    #create target orbit
    o_sf = Orbit.from_body_ephem(Saturn, date_arrive_saturn)
    r_sf, v_sf = o_sf.rv()
    #o_sf.plot()
    if disp_on:
        print('computed saturn')

    ################## Jupiter ################## 

    #guess flyby date of Jupiter
    date_flyby_jupiter = date_arrive_saturn-time_js

    #construct j orbit
    o_j = Orbit.from_body_ephem(Jupiter, date_flyby_jupiter)
    #o_j.plot()

    #compute transfer lambert trajectory
    (v_jo, v_si), = izzo.lambert(Sun.k, o_j.r, o_sf.r, time_js)
    trx_js = Orbit.from_vectors(Sun, o_j.r, v_jo, epoch=date_flyby_jupiter)
    #trx_js.plot()
    if disp_on:
        print('computed J-S')
    ################## Earth ################## 
    #guess flyby date of Earth
    date_flyby_earth = date_arrive_saturn-time_js-time_ej

    #construct j orbit
    o_e2 = Orbit.from_body_ephem(Earth, date_flyby_earth)
    #o_e2.plot()

    #compute transfer lambert trajectory
    (v_eo, v_ji), = izzo.lambert(Sun.k, o_e2.r, o_j.r, time_ej)
    trx_ej = Orbit.from_vectors(Sun, o_e2.r, v_eo, epoch=date_flyby_earth)
    #trx_ej.plot()
    if disp_on:
        print('computed E-J')                            

    ################## Venus2 ################## 
    #guess flyby date of Venus2
    date_flyby_venus2 = date_arrive_saturn-time_js-time_ej-time_ve

    #construct j orbit
    o_v2 = Orbit.from_body_ephem(Venus, date_flyby_venus2)
    #o_v2.plot()

    #compute transfer lambert trajectory
    (v_v2o, v_ei), = izzo.lambert(Sun.k, o_v2.r, o_e2.r, time_ve)
    trx_v2e = Orbit.from_vectors(Sun, o_v2.r, v_v2o, epoch=date_flyby_venus2)
    #trx_v2e.plot()    
    if disp_on:
        print('computed V-E')                            

    ################## Sum delta v ##################                             
    delv_e = norm(v_eo-v_ei)
    delv_j = norm(v_jo-v_ji)
    delv_s = norm(v_si-v_sf)

    total_deltav=sum([delv_e,delv_j,delv_s]) 

    if disp_on:
        print('Total delta-v: ', total_deltav)

    ################## Plot ##################  

    if plot_on:
        op = OrbitPlotter2D()

        op.plot(o_v2,label="Venus2 Orbit")
        op.plot(o_e2,label="Earth2 Orbit")
        op.plot(o_j, label="Jupiter Orbit")
        op.plot(o_sf, label="Saturn Orbit")


        op.plot(trx_v2e, label="V2-E")
        op.plot(trx_ej, label="E-J")
        op.plot(trx_js, label="J-S")
        
    orbits = (o_v2,o_e2,o_j,o_sf)
    trajectories = (trx_v2e,trx_ej,trx_js)
    deltavs = (delv_e,delv_j,delv_s)
    times = (date_flyby_venus2,date_flyby_earth,date_flyby_jupiter,date_arrive_saturn)
    return (total_deltav,orbits,trajectories,deltavs,times)

def print_times(times):
    print("Venus:   ",times[0])
    print("Earth:   ",times[1])
    print("Jupiter: ",times[2])
    print("Saturn:  ",times[3])
    print("**")
    print("V-E: ", (times[1]-times[0]).to(u.year))
    print("E-J: ", (times[2]-times[1]).to(u.year))
    print("E-S: ", (times[3]-times[2]).to(u.year))
        



In [17]:
cassini_js_time.to(u.year).value

3.5017111567419574

In [22]:
t_guess_cassini = [27,cassini_js_time.to(u.year).value,cassini_ej_time.to(u.year).value,cassini_ve_time.to(u.year).value]
sol=trajectory_calculator(t=t_guess_cassini,plot_on=0,disp_on=1)

computed saturn
computed J-S
computed E-J
computed V-E
Total delta-v:  82.09792665510716 km / s


/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/orbit.py:389: TimeScaleWarning:

Input time was converted to scale='tdb' with value 2046-12-31 18:01:09.184. Use Time(..., scale='tdb') instead.

/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/orbit.py:389: TimeScaleWarning:

Input time was converted to scale='tdb' with value 2043-07-01 18:01:09.184. Use Time(..., scale='tdb') instead.

/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/orbit.py:389: TimeScaleWarning:

Input time was converted to scale='tdb' with value 2042-02-16 18:01:09.185. Use Time(..., scale='tdb') instead.

/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/orbit.py:389: TimeScaleWarning:

Input time was converted to scale='tdb' with value 2041-12-23 18:01:09.184. Use Time(..., scale='tdb') instead.



In [ ]:
sol=trajectory_calculator(tmod=[0,0,0,0],plot_on=0,disp_on=1)
if 1:
    op=OrbitPlotter2D()
    op.plot(sol[1][0],label='V')
    op.plot(sol[1][1],label='E')
    op.plot(sol[1][2],label='J')
    op.plot(sol[1][3],label='S')
    op.plot(sol[2][0],label='VE')
    op.plot(sol[2][1],label='EJ')
    op.plot(sol[2][2],label='JS')



/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/orbit.py:389: TimeScaleWarning:

Input time was converted to scale='tdb' with value 2047-03-29 00:01:09.186. Use Time(..., scale='tdb') instead.

/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/orbit.py:389: TimeScaleWarning:

Input time was converted to scale='tdb' with value 2043-09-27 00:01:09.182. Use Time(..., scale='tdb') instead.

/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/orbit.py:389: TimeScaleWarning:

Input time was converted to scale='tdb' with value 2042-05-15 00:01:09.185. Use Time(..., scale='tdb') instead.

/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/orbit.py:389: TimeScaleWarning:

Input time was converted to scale='tdb' with value 2042-03-21 00:01:09.186. Use Time(..., scale='tdb') instead.



computed saturn
computed J-S
computed E-J
computed V-E
Total delta-v:  49.73689056630519 km / s


/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/propagation.py:232: UserWarning:

Frame <class 'astropy.coordinates.builtin_frames.icrs.ICRS'> does not support 'obstime', time values were not returned



In [21]:
sol

(<Quantity 82.09792665510716 km / s>,
 (1 x 1 AU x 24.4 deg (ICRS) orbit around Sun (☉) at epoch 2041-12-23 18:01:09.184 (TDB),
  1 x 1 AU x 23.4 deg (ICRS) orbit around Sun (☉) at epoch 2042-02-16 18:01:09.185 (TDB),
  5 x 5 AU x 23.2 deg (ICRS) orbit around Sun (☉) at epoch 2043-07-01 18:01:09.184 (TDB),
  9 x 10 AU x 22.6 deg (ICRS) orbit around Sun (☉) at epoch 2046-12-31 18:01:09.184 (TDB)),
 (30293036 x -2883088647 km x 24.0 deg (HCRS) orbit around Sun (☉) at epoch 2041-12-23 18:00:00.000 (UTC),
  1 x 10 AU x 23.2 deg (HCRS) orbit around Sun (☉) at epoch 2042-02-16 18:00:00.000 (UTC),
  0 x 47 AU x 27.1 deg (HCRS) orbit around Sun (☉) at epoch 2043-07-01 18:00:00.000 (UTC)),
 (<Quantity 38.076032380696105 km / s>,
  <Quantity 29.27965312341461 km / s>,
  <Quantity 14.742241150996456 km / s>),
 (<Time object: scale='utc' format='iso' value=2041-12-23 18:00:00.000>,
  <Time object: scale='utc' format='iso' value=2042-02-16 18:00:00.000>,
  <Time object: scale='utc' format='iso' val

In [23]:
t_guess_cassini

[27, 3.5017111567419574, 1.3689253935660506, 0.15058179329226556]

In [26]:
bounds = ((27-5,27+5),(0.01,5),(0.01,5),(0.01,1))
options = {'maxiter': 30,'disp': True}

def total_deltav_calculator(tmod):
    print(tmod)
    sol=trajectory_calculator(tmod,plot_on=0,disp_on=0)
    
    return sol[0].to(u.km/u.s).value

opt_sol = opt.minimize(total_deltav_calculator,x0=t_guess_cassini,options=options,bounds=bounds)

opt_sol

[ 27.           3.50171116   1.36892539   0.15058179]
[ 27.00000001   3.50171116   1.36892539   0.15058179]
[ 27.           3.50171117   1.36892539   0.15058179]
[ 27.           3.50171116   1.3689254    0.15058179]
[ 27.           3.50171116   1.36892539   0.1505818 ]
[  3.20000000e+01   1.00000000e-02   1.00000000e-02   1.00000000e+00]
[  3.20000000e+01   1.00000000e-02   1.00000000e-02   1.00000000e+00]
[  3.20000000e+01   1.00000100e-02   1.00000000e-02   1.00000000e+00]
[  3.20000000e+01   1.00000000e-02   1.00000100e-02   1.00000000e+00]
[  3.20000000e+01   1.00000000e-02   1.00000000e-02   1.00000001e+00]
[ 28.75941787   2.27303536   0.89074187   0.44947811]


/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/orbit.py:389: TimeScaleWarning:

Input time was converted to scale='tdb' with value 2046-12-31 18:01:09.184. Use Time(..., scale='tdb') instead.

/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/orbit.py:389: TimeScaleWarning:

Input time was converted to scale='tdb' with value 2043-07-01 18:01:09.184. Use Time(..., scale='tdb') instead.

/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/orbit.py:389: TimeScaleWarning:

Input time was converted to scale='tdb' with value 2042-02-16 18:01:09.185. Use Time(..., scale='tdb') instead.

/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/orbit.py:389: TimeScaleWarning:

Input time was converted to scale='tdb' with value 2041-12-23 18:01:09.184. Use Time(..., scale='tdb') instead.

/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/orbit.py:389: TimeScaleWarning:

Input time was converted to 

[ 28.75941788   2.27303536   0.89074187   0.44947811]
[ 28.75941787   2.27303537   0.89074187   0.44947811]
[ 28.75941787   2.27303536   0.89074188   0.44947811]
[ 28.75941787   2.27303536   0.89074187   0.44947812]
[ 27.416082     3.21114353   1.25584051   0.22126732]
[ 27.41608201   3.21114353   1.25584051   0.22126732]
[ 27.416082     3.21114354   1.25584051   0.22126732]
[ 27.416082     3.21114353   1.25584052   0.22126732]
[ 27.416082     3.21114353   1.25584051   0.22126733]
[ 28.4159158    2.51415369   0.99168391   0.18112024]
[ 28.41591581   2.51415369   0.99168391   0.18112024]


/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/orbit.py:389: TimeScaleWarning:

Input time was converted to scale='tdb' with value 2044-09-23 06:03:52.992. Use Time(..., scale='tdb') instead.

/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/orbit.py:389: TimeScaleWarning:

Input time was converted to scale='tdb' with value 2048-05-31 21:56:14.032. Use Time(..., scale='tdb') instead.

/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/orbit.py:389: TimeScaleWarning:

Input time was converted to scale='tdb' with value 2045-11-25 14:51:57.585. Use Time(..., scale='tdb') instead.

/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/orbit.py:389: TimeScaleWarning:

Input time was converted to scale='tdb' with value 2044-11-28 09:45:53.288. Use Time(..., scale='tdb') instead.

/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/orbit.py:389: TimeScaleWarning:

Input time was converted to 

[ 28.4159158    2.5141537    0.99168391   0.18112024]
[ 28.4159158    2.51415369   0.99168392   0.18112024]
[ 28.4159158    2.51415369   0.99168391   0.18112025]
[ 27.66052213   3.04074292   1.19125931   0.21145213]
[ 27.66052214   3.04074292   1.19125931   0.21145213]
[ 27.66052213   3.04074293   1.19125931   0.21145213]
[ 27.66052213   3.04074292   1.19125932   0.21145213]
[ 27.66052213   3.04074292   1.19125931   0.21145214]
[ 27.83635796   2.91816676   1.14480339   0.20439166]
[ 27.83635797   2.91816676   1.14480339   0.20439166]
[ 27.83635796   2.91816677   1.14480339   0.20439166]
[ 27.83635796   2.91816676   1.1448034    0.20439166]
[ 27.83635796   2.91816676   1.14480339   0.20439167]
[ 27.68051482   3.0268059    1.18597723   0.21064935]
[ 27.68051483   3.0268059    1.18597723   0.21064935]

...

[ 32.           4.52430722   3.80160574   0.1647006 ]
[ 32.           4.52430721   3.80160575   0.1647006 ]
[ 32.           4.52430721   3.80160574   0.16470061]
[ 32.           4.52342153   3.79988291   0.16470676]
[ 32.00000001   4.52342153   3.79988291   0.16470676]
[ 32.           4.52342154   3.79988291   0.16470676]
[ 32.           4.52342153   3.79988292   0.16470676]
[ 32.           4.52342153   3.79988291   0.16470677]
[ 32.           4.5271505    3.79157583   0.15764393]
[ 32.00000001   4.5271505    3.79157583   0.15764393]
[ 32.           4.52715051   3.79157583   0.15764393]
[ 32.           4.5271505    3.79157584   0.15764393]
[ 32.           4.5271505    3.79157583   0.15764394]
[ 32.           4.52409481   3.79838305   0.16343155]
[ 32.00000001   4.52409481   3.79838305   0.16343155]
[ 32.           4.52409482   3.79838305   0.16343155]
[ 32.           4.52409481   3.79838306   0.16343155]
[ 32.           4.52409481   3.79838305   0.16343156]
[ 32.           4.52449952  

/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/orbit.py:389: TimeScaleWarning:

Input time was converted to scale='tdb' with value 2047-06-23 13:48:14.906. Use Time(..., scale='tdb') instead.

/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/orbit.py:389: TimeScaleWarning:

Input time was converted to scale='tdb' with value 2043-09-05 05:10:42.039. Use Time(..., scale='tdb') instead.

/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/orbit.py:389: TimeScaleWarning:

Input time was converted to scale='tdb' with value 2043-07-07 12:32:14.533. Use Time(..., scale='tdb') instead.

/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/orbit.py:389: TimeScaleWarning:

Input time was converted to scale='tdb' with value 2047-06-23 13:48:15.221. Use Time(..., scale='tdb') instead.

/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/orbit.py:389: TimeScaleWarning:

Input time was converted to 

[ 32.00000001   4.52449952   3.79788952   0.16331697]
[ 32.           4.52449953   3.79788952   0.16331697]
[ 32.           4.52449952   3.79788953   0.16331697]
[ 32.           4.52449952   3.79788952   0.16331698]
[ 32.           4.52516642   3.79722668   0.16326178]
[ 32.00000001   4.52516642   3.79722668   0.16326178]
[ 32.           4.52516643   3.79722668   0.16326178]
[ 32.           4.52516642   3.79722669   0.16326178]
[ 32.           4.52516642   3.79722668   0.16326179]


/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/orbit.py:389: TimeScaleWarning:

Input time was converted to scale='tdb' with value 2047-06-23 04:24:37.302. Use Time(..., scale='tdb') instead.

/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/orbit.py:389: TimeScaleWarning:

Input time was converted to scale='tdb' with value 2043-09-05 05:55:16.552. Use Time(..., scale='tdb') instead.

/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/orbit.py:389: TimeScaleWarning:

Input time was converted to scale='tdb' with value 2043-07-07 14:46:06.732. Use Time(..., scale='tdb') instead.

/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/orbit.py:389: TimeScaleWarning:

Input time was converted to scale='tdb' with value 2047-06-23 04:24:37.618. Use Time(..., scale='tdb') instead.

/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/orbit.py:389: TimeScaleWarning:

Input time was converted to 

      fun: 19.204786918664688
 hess_inv: <4x4 LbfgsInvHessProduct with dtype=float64>
      jac: array([ -0.95352988,  -4.63682071,   1.18568479, -21.55289671])
  message: b'STOP: TOTAL NO. of ITERATIONS REACHED LIMIT'
     nfev: 265
      nit: 30
   status: 1
  success: False
        x: array([ 32.        ,   4.52516642,   3.79722668,   0.16326178])

In [27]:
t_guess_cassini

[27, 3.5017111567419574, 1.3689253935660506, 0.15058179329226556]

In [ ]:

options = {'maxiter': 30,'disp': False}
opt_sol = opt.minimize(total_deltav_calculator,x0=opt_sol.x,options=options)
opt_sol

In [28]:
opt_sol

      fun: 19.204786918664688
 hess_inv: <4x4 LbfgsInvHessProduct with dtype=float64>
      jac: array([ -0.95352988,  -4.63682071,   1.18568479, -21.55289671])
  message: b'STOP: TOTAL NO. of ITERATIONS REACHED LIMIT'
     nfev: 265
      nit: 30
   status: 1
  success: False
        x: array([ 32.        ,   4.52516642,   3.79722668,   0.16326178])

In [30]:
sol=trajectory_calculator(t=opt_sol.x,plot_on=0,disp_on=0)

print("Tmod: ",opt_sol.x)
print()
print("Total deltav: ", sol[0])
print("Delta vs: ",[x.value for x in sol[3]])
print()

print('Times:')
print_times(sol[4])
print()
t_ve=sol[4][1]-sol[4][0]
t_ej=sol[4][2]-sol[4][1]
t_js=sol[4][3]-sol[4][2]


trx_ve = sol[2][0]
trx_ej = sol[2][1]
trx_js = sol[2][2]


trx_ve_half = trx_ve.propagate(t_ve/2)
trx_ej_half = trx_ej.propagate(t_ej/2)
trx_js_half = trx_js.propagate(t_ej/2)


op=OrbitPlotter2D()
op.plot(sol[1][0],label='V')
op.plot(sol[1][1],label='E')
op.plot(sol[1][2],label='J')
op.plot(sol[1][3],label='S')
op.plot(trx_ve_half,label='VE')
op.plot(trx_ej_half,label='EJ')
op.plot(trx_js_half,label='JS')

/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/orbit.py:389: TimeScaleWarning:

Input time was converted to scale='tdb' with value 2052-01-01 00:01:09.184. Use Time(..., scale='tdb') instead.

/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/orbit.py:389: TimeScaleWarning:

Input time was converted to scale='tdb' with value 2047-06-23 04:24:37.302. Use Time(..., scale='tdb') instead.

/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/orbit.py:389: TimeScaleWarning:

Input time was converted to scale='tdb' with value 2043-09-05 05:55:16.552. Use Time(..., scale='tdb') instead.

/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/orbit.py:389: TimeScaleWarning:

Input time was converted to scale='tdb' with value 2043-07-07 14:46:06.732. Use Time(..., scale='tdb') instead.

/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/propagation.py:232: UserWarning:

Frame <class 'astropy.coord

Tmod:  [ 32.           4.52516642   3.79722668   0.16326178]

Total deltav:  19.204786918664688 km / s
Delta vs:  [0.5790827206586804, 8.43114315329468, 10.194561044711325]

Times:
Venus:    2043-07-07 14:44:57.548
Earth:    2043-09-05 05:54:07.370
Jupiter:  2047-06-23 04:23:28.118
Saturn:   2052-01-01 00:00:00.000
**
V-E:  0.16326177595941857 yr
E-J:  3.7972266822614382 yr
E-S:  4.525166422109325 yr



FigureWidget({
    'data': [{'hoverinfo': 'none',
              'line': {'color': 'rgb(31, 119, 180)', 'dash':…

In [36]:
tmod = opt_sol.x
tmod = [27.238877,4.798,1.2879,0.1313]
sol=trajectory_calculator(t=tmod,plot_on=0,disp_on=0)

print("Tmod: ",opt_sol.x)
print()
print("Total deltav: ", sol[0])
print("Delta vs: ",[x.value for x in sol[3]])
print()

print('Times:')
print_times(sol[4])
print()
t_ve=sol[4][1]-sol[4][0]
t_ej=sol[4][2]-sol[4][1]
t_js=sol[4][3]-sol[4][2]


trx_ve = sol[2][0]
trx_ej = sol[2][1]
trx_js = sol[2][2]


trx_ve_half = trx_ve.propagate(t_ve/2)
trx_ej_half = trx_ej.propagate(t_ej/2)
trx_js_half = trx_js.propagate(t_ej/2)


op=OrbitPlotter2D()
op.plot(sol[1][0],label='V')
op.plot(sol[1][1],label='E')
op.plot(sol[1][2],label='J')
op.plot(sol[1][3],label='S')
op.plot(trx_ve_half,label='VE')
op.plot(trx_ej_half,label='EJ')
op.plot(trx_js_half,label='JS')

/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/orbit.py:389: TimeScaleWarning:

Input time was converted to scale='tdb' with value 2047-03-29 00:00:54.001. Use Time(..., scale='tdb') instead.

/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/orbit.py:389: TimeScaleWarning:

Input time was converted to scale='tdb' with value 2042-06-10 12:44:49.200. Use Time(..., scale='tdb') instead.

/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/orbit.py:389: TimeScaleWarning:

Input time was converted to scale='tdb' with value 2041-02-25 03:00:56.161. Use Time(..., scale='tdb') instead.

/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/orbit.py:389: TimeScaleWarning:

Input time was converted to scale='tdb' with value 2041-01-08 04:02:23.279. Use Time(..., scale='tdb') instead.

/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/propagation.py:232: UserWarning:

Frame <class 'astropy.coord

Tmod:  [ 32.           4.52516642   3.79722668   0.16326178]

Total deltav:  6.010478813049802 km / s
Delta vs:  [0.19207023597361006, 0.0776495284180622, 5.74075904865813]

Times:
Venus:    2041-01-08 04:01:14.095
Earth:    2041-02-25 02:59:46.975
Jupiter:  2042-06-10 12:43:40.015
Saturn:   2047-03-28 23:59:44.815
**
V-E:  0.1313 yr
E-J:  1.2879 yr
E-S:  4.798 yr



FigureWidget({
    'data': [{'hoverinfo': 'none',
              'line': {'color': 'rgb(31, 119, 180)', 'dash':…

In [ ]:
t_ve

In [ ]:
trx_ve = sol[2][0]

In [ ]:
import poliastro.twobody.propagation as Propagation

In [ ]:
trx_ve_half = trx_ve.propagate(t_ve/2)

In [ ]:
trx_ve_half.plot()

In [ ]:
norm(o_e.v.to(u.km/u.s))

In [ ]:
print_times(sol[4])

In [ ]:
o_e

In [34]:
(Time("2047-03-29", scale="utc")-Time("2020-01-01", scale="utc")).to(u.year)

<Quantity 27.238877481177276 yr>